# Cartesian Product


## Cartesian Product

So far we have restricted ourselves to operators that operate on one table at a time.  This is logical in the sense that our operators create relations!  However, we know that a typical database contains many tables, which in fact may be related.  So, how do we do queries using mulitple tables?  

The first step toward applying the operators we have learned about so far to multiple tables is to merge the tables together   We do this using the cartesian product.   A cartesian product creates one table out of two tables by creating every possible combination of each row in table A with each row in table B, forming a new relation with A+B columns, and A*B rows!

![](cartprod1.png)


Of course this can create an **enormous** table, so the cartesian product is always followed by a query where we limit the number of rows by comparing a column in relation A against a column in relation B.

![](cartprod2.png)


# Natural Join

The natural join or ``njoin`` operator takes the pattern of cartesian product followed by query, and wraps it all into one operation subject to the following:

* The query condition tests for equality
* The query condition of equality applies to all columns with the same name in both relations.

You can see this in the following diagram, where we have two relations.  Both have a column named C1.  
The resulting relation has a single C1 column where only the rows where C1 holds the same value in both relations.  The other values from the row are filled in with the values from the matching rows.

![](njoin.png)



In [8]:
import warnings
warnings.filterwarnings('ignore')

from reframe import Relation
r1 = Relation('/home/faculty/millbr02/pub/R1.csv',sep=',')
r2 = Relation('/home/faculty/millbr02/pub/R2.csv',sep=',')
r1

,C1,C2
0,1,A
1,2,B
2,3,C
3,5,D


In [9]:
r2

,C1,C3
0,7,E
1,3,F
2,1,J
3,2,L


In [10]:
r1.njoin(r2)

,C1,C2,C3
0,1,A,J
1,2,B,L
2,3,C,F


In [11]:
%load_ext sql

In [16]:
%sql postgresql://millbr02:@localhost/jtest

'Connected: millbr02@jtest'

In [18]:
%%sql

select * 
from r1 natural join r2


3 rows affected.


C1,C2,C3
1,A,J
2,B,L
3,C,F
